In [19]:
import ee
import geemap

In [20]:

# Authenticate and initialize GEE
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# Load the MTBS burn severity dataset
mtbs = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")

In [ ]:
# Filter the dataset by date range
mtbs_filtered = mtbs.filterDate('2016-01-01', '2023-12-31')

In [ ]:
# Example AOI: California
aoi = ee.Geometry.Rectangle([-124.4, 32.5, -114.1, 42.0])  # Bounding box for California

In [ ]:
# Define visualization parameters
vis_params = {
    'bands': ['Severity'],
    'min': 1,
    'max': 4,
    'palette': ['white', 'green', 'yellow', 'red']  # Colors for unburned, low, moderate, and high severity
}

In [24]:
# Create a map
map_ = geemap.Map(center=[37.5, -119], zoom=6)
map_.addLayer(mtbs.median().clip(aoi), vis_params, "MTBS Burn Severity 2016-2023")
map_.addLayer(aoi, {}, "AOI")

# Define legend details based on the class table
legend_dict = {
    'Background': '#000000',
    'Unburned to Low': '#006400',
    'Low': '#7fffd4',
    'Moderate': '#ffff00',
    'High': '#ff0000',
    'Increased Greenness': '#7fff00',
    'Non-Mapping Area': '#ffffff'
}

# Add the legend to the map
map_.add_legend(title="MTBS Burn Severity Legend", legend_dict=legend_dict)

# Display the map
map_

Map(center=[37.5, -119], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [23]:
# Load the MTBS burned area boundaries mtbs_boundaries
mtbs_boundaries = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

# Define the AOI (Example: California)
aoi = ee.Geometry.Rectangle([-124.4, 32.5, -114.1, 42.0])  # Bounding box for California

# Filter the mtbs_boundaries by date range (2016-01-01 to 2021-12-31) and clip to the AOI
mtbs_boundaries_filtered = mtbs_boundaries.filterBounds(aoi)

# Visualization parameters
visparams = {
    'fillColor': '#ff8a50',  # Fill color
    'color': '#ff5722',      # Outline color
    'width': 1.0             # Outline width
}

# Create a map centered on California
map_ = geemap.Map(center=[37.5, -119], zoom=6)

# Add the filtered and clipped burned area boundaries layer to the map
map_.addLayer(mtbs_boundaries_filtered, visparams, 'MTBS Burned Area Boundaries (2016-2021)')

# Add the AOI boundary to the map for reference
map_.addLayer(aoi, {'color': 'blue'}, 'AOI Boundary')

# Display the map
map_

Map(center=[37.5, -119], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [28]:

# Load the MTBS burned area boundaries dataset
dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

# Define the field and value to filter by (e.g., 'Event_ID' or 'Incid_Name')
field_name = 'Event_ID'  # You can change this to 'Incid_Name' or another field
field_value = 'CA3983912034520210702'  # Replace with the specific value you're looking for

# Filter the dataset by the specified field and value
filtered_feature = dataset.filter(ee.Filter.eq(field_name, field_value))

# Check if the filtered dataset is empty
count = filtered_feature.size().getInfo()
if count == 0:
    print(f"No feature found with {field_name}: {field_value}")
else:
    print(f"Displaying feature with {field_name}: {field_value}")

# Visualization parameters
vis_params = {
    'fillColor': '#ff8a50',  # Fill color for the boundary
    'color': '#ff5722',      # Outline color
    'width': 2.0             # Outline width
}

# Create a map object
map_ = geemap.Map()

# Add the filtered feature to the map
map_.addLayer(filtered_feature, vis_params, f"{field_name}: {field_value}")

# Zoom to the feature if it exists
if count > 0:
    map_.centerObject(filtered_feature, zoom=10)

# Display the map
map_

Displaying feature with Event_ID: CA3983912034520210702


Map(center=[39.93642291484538, -120.17855459031658], controls=(WidgetControl(options=['position', 'transparent…

In [33]:
import pandas as pd
# Load the MTBS burned area boundaries dataset
dataset = ee.FeatureCollection('USFS/GTAC/MTBS/burned_area_boundaries/v1')

# Define the Event_ID to filter by
event_id = 'CA3983912034520210702'

# Filter the dataset by the specific Event_ID
filtered_feature = dataset.filter(ee.Filter.eq('Event_ID', event_id)).first()

# Check if the feature exists
if filtered_feature is None:
    print(f"No feature found with Event_ID: {event_id}")
else:
    # Get the feature's properties
    properties = filtered_feature.getInfo()['properties']

    # # Create a dictionary with the specific properties of interest
    # selected_properties = {
    #     'Event_ID': properties.get('Event_ID'),
    #     'Incid_Name': properties.get('Incid_Name'),
    #     'BurnBndAc': properties.get('BurnBndAc'),
    #     'Incid_Type': properties.get('Incid_Type'),
    #     'irwinID': properties.get('irwinID')
    # }

    # Convert the selected properties dictionary to a DataFrame
    df = pd.DataFrame([properties])


In [34]:
df

,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,Comment,Event_ID,High_T,Ig_Date,IncGreen_T,Incid_Name,...,Map_ID,Map_Prog,Mod_T,NoData_T,Perim_ID,Post_ID,Pre_ID,dNBR_offst,dNBR_stdDv,irwinID
0,Initial,108368,39.957,-120.181,CA3987512036820210703 is a duplicate of this f...,CA3983912034520210702,650,1625209200000,-150,SUGAR,...,10024011,MTBS,450,-970,,804303220210805,804303220200701,44,32,33088F36-F802-4500-86A4-82DA6A62425F
